# Brest Cancer Wisconsin Dataset Example

This Jupyter Notebook demonstrates how to build and use a neural network with dense layers to process and classify data.

In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo

from src.preprocessing import DataLoader, min_max_scaler, train_test_split
from src.loss import BinaryCrossentropy
from src.encode import BinaryEncoder
from src.optimizer import Adam
from src.scheduler import CosineScheduler
from src.tensor import Tensor
from src.structure import Layer, Dense, Dropout, LeakyRelu, Sigmoid

In [ ]:
# Dataset is small, so it's faster to use CPU
Tensor.set_default_device("cpu")

breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

X = X.to_numpy()
y = y.to_numpy(dtype=str)

In [ ]:
classes = tuple(np.unique(y))

# Normalize data
data = min_max_scaler(X, -1, 1)

# Split data into training and testing sets
data_train, excepted_train, data_test, expected_test = train_test_split(data, y, random_state=0)

In [ ]:
class Model(Layer):
    def __init__(self) -> None:
        self.layers = [
            Dense(len(X[0])),
            LeakyRelu(),
            Dense(32),
            LeakyRelu(),
            Dropout(0.2),
            Dense(16),
            LeakyRelu(),
            Dense(8),
            LeakyRelu(),
            Dense(1),
            Sigmoid(),
        ]

        self.encoder = BinaryEncoder(classes)

    def __call__(self, inputs: Tensor) -> Tensor:
        return inputs.sequential(self.layers)

    def train(self, steps: int) -> None:
        data = DataLoader(data_train, excepted_train, batch_size=4)

        opt = Adam(list(self.parameters), CosineScheduler(learning_rate=1e-2, min_lr=1e-9, max_steps=100, cyclic=True))

        loss_func = BinaryCrossentropy()

        for _ in range(steps):
            x, y = next(data)

            opt.zero_grad()

            loss = loss_func(self(x), self.encoder(y))
            loss.backward()

            opt.step()

            print(f"loss: {loss.mean()}")

    @Tensor.no_grad()
    def evaluate(self, data: Tensor, expected: Tensor) -> float:
        correct = 0
        total = 0

        for i in range(len(data)):
            total += 1

            if self.encoder.decode(self(data[i])) == expected[i].item():
                correct += 1

        return correct / total

nn = Model()

nn.train(len(data_train) * 4)

print(f"test: {nn.evaluate(data_test, expected_test)},train: {nn.evaluate(data_train, excepted_train)}")